In [1]:
import os
from recognition.compare import compareFeatures
from recognition.recognition import extractPhotoFeatures

from backend.database.database import SessionLocal
from backend.database.crud import create_outfit, get_all_outfits, get_user, create_user
from backend.database.schemas import OutfitCreate, UserCreate
from sqlalchemy.orm import Session

TEST_USER_ID = 1

def setUpTestUser(db):
    if not get_user(db, TEST_USER_ID):
        create_user(db, UserCreate(email="test@mail.com", password="test"))

def setUpDB(sharedDBPath, db):
    sharedDB_filenames = []
    numEntries = []
    for filename in os.listdir(sharedDBPath):
        filePath = os.path.join(sharedDBPath, filename)
        entriesCount=0
        print("for filename in sharedDBPath, ", filename)
        if os.path.isfile(filePath):
            vectors = extractPhotoFeatures(filePath)
            for v in vectors:
                outfit = OutfitCreate(
                    name=v["name"],
                    description="A comfortable and stylish outfit",
                    features=v,
                )
                print("adding", v["name"])
                entriesCount+=1
                create_outfit(db, outfit, TEST_USER_ID)
                numEntries.append(entriesCount)
                sharedDB_filenames.append(filename)
    return sharedDB_filenames, numEntries

def getTestPhotos(testPath):
    test_filenames = []
    numEntries = []
    vectors=[]
    for filename in os.listdir(testPath):
        filePath = os.path.join(testPath, filename)
        entriesCount=0
        print("for filename in testPath, ", filename) 
        if os.path.isfile(filePath):
            vectorsCurrent = extractPhotoFeatures(filePath)
            vectors.append(vectorsCurrent)
            for v in vectorsCurrent:
                print("adding", v["name"])
                entriesCount+=1
                numEntries.append(entriesCount)
                test_filenames.append(filename)
    return vectors, test_filenames, numEntries

def visualize_results(vectors, outfits):
    results=[]
    for vector in vectors:
        for v in range(len(vector)):
            #print(vector[v])
            for outfit in outfits:
                outfitVector = outfit.features
                results.append(compareFeatures(vector[v], outfitVector))
    return results



def deleteDB(database_path):
    try:
        if os.path.exists(database_path):
            os.remove(database_path)
            print(f"Database file at {database_path} successfully deleted.")
            return True
        else:
            print(f"Database file at {database_path} does not exist.")
            return False
    except Exception as e:
        print(f"Error deleting database file: {str(e)}")
        return False


In [2]:
#Get sharedDB photos, extract objects and features
db: Session = SessionLocal()
setUpTestUser(db)
sharedFileNames, numsShared = setUpDB("./test/sharedDB", db)



for filename in sharedDBPath,  0.jpg
adding Pants
adding Top
adding Shoe
adding Shoe
for filename in sharedDBPath,  1.png
adding Pants
adding Top
adding Hat
adding Shoe
for filename in sharedDBPath,  2.jpg
adding Pants
adding Top
adding Shoe
adding Shoe


In [3]:
#Get all outfits from sharedDB
outfits = get_all_outfits(db, TEST_USER_ID, 1000)

#Get all outfits from test/tests
vectors, testFileNames, numsTest = getTestPhotos("./test/tests")

results = visualize_results(vectors, outfits)


for filename in testPath,  2.jpg
adding Pants
adding Top
adding Shoe
adding Shoe
for filename in testPath,  3.jpg
adding Pants
adding Top
adding Shoe
adding Shoe
for filename in testPath,  4.jpg
adding Pants
adding Top


In [4]:
import csv 


with open('output_TEST.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Test Image", "Shared Image", "Test Image Name", "Shared Image Name", "Similarity"])
    testIndex = 0
    sharedIndex = 0
    current=0
    for vector in vectors:
        for v in range(len(vector)):
            #print("test vector: ", curVector["name"])
            for outfit in outfits:
                outfitVector = outfit.features
                #print(numsTest[testIndex], " ", numsShared[sharedIndex])
                #print(curVector["name"])
                writer.writerow([testFileNames[testIndex], sharedFileNames[sharedIndex], vector[v]["name"], outfitVector["name"], results[current]])
                #print("shared db vector: ", outfitVector["name"])
            
                sharedIndex+=1
                current+=1
            sharedIndex=0
            testIndex+=1



In [ ]:

################ NOTHING ####################
i=0
results = visualize_results(vectors, outfits)
testIndex = 0
sharedIndex = 0
for vector in vectors:
    for v in range(len(vector)):
        print("test vector: ", vector[v]["name"])
        for outfit in outfits:
            outfitVector = outfit.features
            print(numsTest[testIndex], " ", numsShared[sharedIndex])
            print("shared db vector: ", outfitVector["name"])
        
            sharedIndex+=1
            i+=1
        sharedIndex=0
        testIndex+=1


print(i)


print(numsShared)
print(numsTest)
print(sharedFileNames)
print(testFileNames)
